In [27]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os 
import sys
import mediapipe as mp



#read img 0001.jpg
#檢查有沒有讀到圖片
if not os.path.exists('0001.jpg'):
    print("image not found!")
    sys.exit()
img=cv2.imread('0001.jpg')
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [28]:
hands = mp.solutions.hands.Hands()
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#將當前frame讀取並用mediapipe抓出keypoint
results = hands.process(img)

# 如果有抓到keypoint
if results.multi_hand_landmarks:
    # print("keypoint detected!")
    keypoint_2hands=[] #(2,21,3)
    keypoint_score_2hands = [] #(2,21)
    # results.multi_hand_landmarks共兩隻手 
    for index,hand_landmarks in enumerate(results.multi_hand_landmarks[:2]):
        keypoint_21 = [] #(21,3)
        keypoint_score_21 = [] #(21)
        
        #整隻手的信心值
        confidence_score = results.multi_handedness[index].classification[0].score
        # print(f'hand#{index} confidence_score:{confidence_score}')
        
        for idx,landmark in enumerate(hand_landmarks.landmark):#21
            # 抓到骨架在frame圖上的座標
            x_cord = int(round(landmark.x * img.shape[1]))
            y_cord = int(round(landmark.y * img.shape[0]))
            # 畫在frame上
            #show出圖片並等待使用者按下任意鍵
            cv2.imshow('img', img)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            
            # depth_value = int(depthIMG[y_cord, x_cord])
            # 紀錄每個關節的座標
            current_keypoint = [landmark.x, landmark.y, landmark.z]
            #current_keypoint shape : (3,) 
            # print(f'current_keypoint shape:{np.array(current_keypoint).shape}')
            #如果index為 {7,11,15,19} 則不append
            if idx in [7,11,15,19]:
                cv2.circle(img, (x_cord, y_cord), 5, (0, 255, 0), -1)
                pass
            else:
                cv2.circle(img, (x_cord, y_cord), 5, (0, 0, 255), -1)
                
                keypoint_21.append(current_keypoint) #3
                # print(f'keypoint_21 shape:{np.array(keypoint_21).shape}')
                # 紀錄每個關節的信心值
                # mediapipe的信心值介於0~1之間，越接近1代表越有信心
                # confidence_score = landmark.visibility
                keypoint_score_21.append(confidence_score)
                # print(f'_confidence_score:{confidence_score}')
            
        keypoint_2hands.append(np.array(keypoint_21, dtype=np.float16)) # 每個frame的keypoints (T, 21, 3) =(frame, keypoint, channel(x,y,z))
        keypoint_score_2hands.append(np.array(keypoint_score_21, dtype=np.float16))
        
    print(f'keypoint_2hands shape:{np.array(keypoint_2hands).shape}')
    # make sure keypoint_2hands shape is (2,21,3)
    # 如果只有一隻手，則填補一隻空手
    if len(keypoint_2hands) == 1:
        keypoint_2hands.append(np.zeros((21,3)))
        keypoint_score_2hands.append(np.zeros((21))) 
        print(f'補完後 keypoint_2hands shape: {np.array(keypoint_2hands).shape} ')
    elif len(keypoint_2hands) == 0:# 補2隻空手
        keypoint_2hands.append(np.zeros((21,3)))
        keypoint_score_2hands.append(np.zeros((21)))
        keypoint_2hands.append(np.zeros((21,3)))
        keypoint_score_2hands.append(np.zeros((21)))
        print(f'補完後 keypoint_2hands shape: {np.array(keypoint_2hands).shape} ')
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("no keypoint detected!")
    pass


keypoint_2hands shape:(2, 17, 3)
